In [ ]:
#Again we use the "!" command to specify we want to execute commands via the terminal, outside python
#rootls is an example of a root command line tool: 
#https://root.cern/manual/storing_root_objects/#root-command-line-tools
#Allows to run common operations from the linux/mac terminal, without running ROOT or python directly.
! rootls ../Lecture1/tree2.root

In [ ]:
#To do the same in python we would have to type:
from ROOT import TFile
myFile = TFile.Open("../Lecture1/tree2.root")
#Note the output is a bit more detailed though
myFile.ls()

In [ ]:
#"make" is a standard command to build an executable, which you can run. An executable can be thought of as a 
#translation of your computer code into something the computer can understand. When using python this happens in
#real time, behind the scenes.
#The "!" tells Jupyter to run the command in the underlying terminal.
! make 
#You can see the commands "make" actually runs printed below.
#Lots of options passed to g++ (don't worry about these) - the important part is it creates library files (*.d,*.o)
#and eventually an executable which you run ("myROOTCommand")

In [ ]:
#Run the command in the terminal, and we see what it does.
! ./myROOTCommand ../Lecture1/tree2.root t2

In [ ]:
#We can also make a python shortcut command!
! ./myROOTcommand.py ../Lecture1/tree2.root t2

In [ ]:
#Task 1
#Make your own python shortcut command which draws an arbitrary variable from the TTree in tree2.root

In [ ]:
#Now lets use uproot - note there is no c++ version! Python only.
import uproot
#Load the TTree from the ATLAS open data (note the miniTree variable is NOT of type TTree - we are now
#in the uproot ecosystem and not in the ROOT ecosystem
miniTreeZmumu = uproot.open("mc_147771.Zmumu.root")["mini"]
#Lets print the variables in the TTree
miniTreeZmumu.keys()
#note these variables are defined at http://opendata.atlas.cern/books/current/openatlasdatatools/_book/variable_names.html

In [ ]:
#choose which variables we want to use
#flatten = false is needed if some variables are arrays - for example here we have one value of jet_n for each
#collision event, but many values of jet_pt for each collision event.
#Notice that the size of the jet_pt array corresponds to alljet_n (NOT jet_N!)
df_Zmumu = miniTreeZmumu.pandas.df(["alljet_n","jet_pt"],flatten=True,entrystop=1000)
df_Zmumu

In [ ]:
df_Zmumu.hist()

In [ ]:
miniTreeTTbar = uproot.open("mc_117049.ttbar_had.root")["mini"]
df_ttbar = miniTreeTTbar.pandas.df(["alljet_n","jet_pt"],flatten=True,entrystop=1000)
df_ttbar.hist("jet_pt",bins=100,range=[0,200000])
df_ttbar.hist("alljet_n",bins=10)

In [ ]:
df_Zmumu=df_Zmumu.assign(signal=1)
df_ttbar=df_ttbar.assign(signal=0)
df_list = [df_Zmumu,df_ttbar]
import pandas
df_all = pandas.concat(df_list)
df_all

In [ ]:
df_all.hist("jet_pt",bins=100,range=[0,200000])

In [ ]:
df_all.hist("signal")

In [ ]:
from keras.layers import Dense
from keras.models import Sequential
model = Sequential()
model.add(Dense(12,input_dim=2,activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
dataset = df_all.values
X = dataset[:,0:2]
Y = dataset[:,2]

In [ ]:
model.compile(loss="binary_crossentropy",optimizer='adam',metrics=['accuracy'])
model.fit(X,Y,epochs=150,batch_size=10)

In [ ]:
scores = model.evaluate(X,Y)

In [ ]:
print("\n%s: %.2f%%" % (model.metrics_names[1],scores[1]*100))

In [ ]:
predictions = model.predict(X)
print([round(x[0]) for x in predictions])